<a href="https://colab.research.google.com/github/amelliaaas/PraktikumML_282-305/blob/main/TugasPraktikumModul3A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Define kaggle json path
# Sesuaikan dengan path anda
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/TugasPraktikum"

In [3]:
# Ubah lokasi direktori kerja
# Sesuaikan dengan path anda
%cd /content/drive/My Drive/TugasPraktikum

/content/drive/My Drive/TugasPraktikum


In [4]:
# Cek apakah api sudah terbaca oleh sistem
!ls  

Covid19-dataset  dataset  kaggle.json


In [5]:
!kaggle datasets download -d pranavraikokte/covid19-image-dataset

 99% 157M/158M [00:01<00:00, 123MB/s]
100% 158M/158M [00:01<00:00, 125MB/s]


In [5]:
# Cek isi direktori kerja dan memastikan dataset telah berhasil didownload.
!ls

Covid19-dataset  dataset  kaggle.json


In [7]:
# Ekstrak file dataset dan hapus file zip dataset agar tidak memakan banyak tempat.
!unzip \*.zip &> /dev/null && rm *.zip

In [8]:
# Cek isi direktori kerja untuk memastikan dataset telah berhasil diekstrak.
!ls

Covid19-dataset  kaggle.json


In [9]:
#Buat folder baru bernama dataset
!mkdir dataset

In [23]:
!mkdir Covid19-dataset

In [6]:
base_dir = 'Covid19-dataset'

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

train_covid_dir = os.path.join(train_dir, 'Covid')
train_normal_dir = os.path.join(train_dir, 'Normal')
train_pneumonia_dir = os.path.join(train_dir, 'Viral Pneumonia')

test_covid_dir = os.path.join(test_dir, 'Covid')
test_normal_dir = os.path.join(test_dir, 'Normal')
test_pneumonia_dir = os.path.join(test_dir, 'Viral Pneumonia')


In [17]:
#moving all dataset to training
import shutil

dataset1= os.listdir(test_covid_dir)
for file in dataset1:
  new_path = shutil.move(f"{test_covid_dir}/{file}",train_covid_dir)

dataset2= os.listdir(test_normal_dir)
for file in dataset2:
  new_path = shutil.move(f"{test_normal_dir}/{file}",train_normal_dir)

dataset3= os.listdir(test_pneumonia_dir)
for file in dataset3:
  new_path = shutil.move(f"{test_pneumonia_dir}/{file}",train_pneumonia_dir)



In [18]:
!mkdir Covid19-dataset/datasets

In [19]:
import splitfolders
splitfolders.ratio(train_dir, output="Covid19-dataset/datasets", seed=1337, ratio=(.8, .19, .01))

Copying files: 312 files [00:06, 46.78 files/s]


In [21]:
# Definisikan path untuk data train test dan validation
# sesuaikan dengan path anda
import os
base_dir = 'Covid19-dataset/datasets'
train_path = os.path.join(base_dir, 'train')
test_path  = os.path.join(base_dir, 'test')
val_path   = os.path.join(base_dir, 'val')

In [22]:
# TULIS KODE ANDA DISINI

from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle

batch_size = 32
augmen_gen = ImageDataGenerator(rescale=1. / 255,
                                rotation_range=40,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                fill_mode='nearest')

train_gen = augmen_gen.flow_from_directory(
    train_path,
    class_mode = 'categorical',
    shuffle=True,
    target_size = (224,224),
    batch_size=batch_size,
    color_mode ='rgb'
)

val_gen = augmen_gen.flow_from_directory(
    val_path,
    class_mode = 'categorical',
    shuffle=True,
    target_size = (224,224),
    batch_size= batch_size,
    color_mode ='rgb'
)

test_gen = augmen_gen.flow_from_directory(
    test_path,
    class_mode = 'categorical',
    shuffle=False,
    target_size = (224,224),
    batch_size=batch_size,
    color_mode ='rgb',
)

Found 249 images belonging to 3 classes.
Found 59 images belonging to 3 classes.
Found 4 images belonging to 3 classes.


In [23]:
# IMPORT SEMUA PAKET YANG ANDA PERLUKAN UNTUK MENDEFINISIKAN MODEL DISINI

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Conv2D, MaxPool2D, Flatten, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
import time

In [24]:
# DEFINISIKAN MODEL ANDA DISINI

#Feature Selection
model = Sequential()

model.add(InputLayer(input_shape=[224,224,3]))
model.add(Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))
model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))
model.add(Conv2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Flatten())

#Fully Connected Layer
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))


In [25]:
# TULIS CODE UNTUK MENAMPILKAN SUMMARY DARI ARSITEKTUR MODEL ANDA DISINI

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 28, 28, 128)       0

In [26]:
# TULIS CODE CALLBACKS ANDA DISINI

best_model_path = 'best_model.h5'
checkpoint_callback = ModelCheckpoint(best_model_path,
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      verbose=1)
reduce_callback = ReduceLROnPlateau(monitor='val_accuracy', patience=3, factor=0.5, min_lr=0.000003, verbose=1)
callbacks_list = [checkpoint_callback, reduce_callback]

In [27]:
# TULIS CODE UNTUK DEFINISI LEARNING RATE DISINI 

learning_rate = 0.00001

In [28]:
# TULIS CODE UNTUK TRAINING DISINI

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=learning_rate),
              metrics='accuracy')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [29]:
# TULIS CODE UNTUK MENCETAK LAMA TRAINING TIME DISINI

start = time.time()
H = model.fit_generator(train_gen, epochs=100,validation_data=val_gen, verbose=1, callbacks=[callbacks_list])

End = time.time()
print ("training time: ", End-start,"s")  # the time would be round to 3 decimal in seconds

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
8/8 [==============================] - 51s 6s/step - loss: 1.1193 - accuracy: 0.4578 - val_loss: 1.0863 - val_accuracy: 0.3729

Epoch 00001: val_accuracy improved from -inf to 0.37288, saving model to best_model.h5
Epoch 2/100
8/8 [==============================] - 45s 5s/step - loss: 0.6954 - accuracy: 0.7229 - val_loss: 1.0586 - val_accuracy: 0.3220

Epoch 00002: val_accuracy did not improve from 0.37288
Epoch 3/100
8/8 [==============================] - 43s 5s/step - loss: 0.7277 - accuracy: 0.6305 - val_loss: 1.0346 - val_accuracy: 0.4576

Epoch 00003: val_accuracy improved from 0.37288 to 0.45763, saving model to best_model.h5
Epoch 4/100
8/8 [==============================] - 43s 5s/step - loss: 0.6929 - accuracy: 0.6908 - val_loss: 1.0291 - val_accuracy: 0.4237

Epoch 00004: val_accuracy did not improve from 0.45763
Epoch 5/100
8/8 [==============================] - 43s 5s/step - loss: 0.5669 - accuracy: 0.7751 - val_loss: 1.0315 - val_accuracy: 0.3390

Epoch 00005: 

In [1]:
# IMPORT SEMUA PAKET YANG ANDA PERLUKAN UNTUK MENGEVALUASI MODEL DISINI

%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
from google.colab import files

In [6]:
# TULIS CODE UNTUK EVALUASI MODEL DISINI

model.load_weights(best_model.h5_path)

ypred = model.predict(test_gen)
prediction = np.argmax(ypred, axis=1)
print(classification_report(test_gen.classes, prediction))

NameError: ignored

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Actual')
    plt.xlabel('Predicted')

cnf_matrix=confusion_matrix(test_gen.classes,prediction)
plot_confusion_matrix(cnf_matrix, classes=test_gen.class_indices)

In [ ]:
# TULIS CODE UNTUK PLOT EVALUASI MODEL DISINI

plt.style.use("ggplot")

plt.plot(np.arange(0, 25), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 25), H.history["val_loss"], label="val_loss")
plt.title("Loss Plot")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.show()

plt.plot(np.arange(0, 25), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, 25), H.history["val_accuracy"], label="val_acc")
plt.title("Accuracy Plot")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
import PIL

def predictModel(image_path):
    img = PIL.Image.open(image_path)
    img_resized = img.resize((224, 224), PIL.Image.LANCZOS)

    plt.imshow(img_resized)

    img_array = np.expand_dims(np.array(img_resized), axis=0)

    pred_coba = model.predict_classes(img_array)
    a = model.predict(img_array)

    labels = np.argmax(a, axis=1)

    plt.title(f"Pred : {a}\nLabels : {labels}")

predictModel("/content/drive/My Drive/TugasPraktikum/Covid19-dataset/test/Covid/051.jpeg")